In [1]:
import pandas as pd
import httplib2
from pathlib import Path
from bs4 import BeautifulSoup, SoupStrainer
pd.set_option('display.max_rows', 200)

In [2]:
def get_data():
    """
    The link to the Sponsor list  is updated. This function is designed to fetch the newest version
    """
    # Get webpage
    http = httplib2.Http()
    status, response = http.request('https://www.gov.uk/government/publications/register-of-licensed-sponsors-workers')
    
    # Search for the link
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href') and link['href'].startswith("https://assets.publishing.service.gov.uk/"):
            file_name = link['href'].split('/')[-1]
            print("\nLast updated:", file_name[:10])
            
            # Fetch the latest data, cache and use old data if there is no update
            if Path(file_name).is_file():
                df = pd.read_csv(file_name)
            else:
                df = pd.read_csv(link['href'])
                df.to_csv(file_name)
                
    return df

In [3]:
# Fetch data
data = get_data()


Last updated: 2023-03-17


In [19]:
def search(word, based_on=None, route=None):
    
    if based_on is None:
        based_on = "org"
    if route is None:
        route = "Skilled Worker"
        
    """
    based_on: Use "org", "toc" or "c" to search based on "Organisation Name", "Town/City" or "County" repectively. 
    word: This is the text to be searched
    route: Any of the elements present in data['Route'].unique(). default is "Skilled Worker".
    
    returns: A dataframe showing the results of the search.
    """
    
    expand_arg = {'org': "Organisation Name", 'toc': "Town/City", 'c': "County"}
    column = expand_arg[based_on]
    df = data.dropna(subset=[column])
    sponsor = df[df[column].str.contains(word, case=False)]
    return sponsor[sponsor["Route"] == route]

In [28]:
response = search(word="University of E", based_on="org", route="Skilled Worker")
response

,Organisation Name,Town/City,County,Type & Rating,Route
68351,University of East Anglia,Norwich,Norfolk,Worker (A rating),Skilled Worker
68353,University of East London,London,NaN,Worker (A rating),Skilled Worker
68355,University of Edinburgh,Edinburgh,NaN,Worker (A (SME+)),Skilled Worker
68359,University of Essex,Colchester,Essex,Worker (A rating),Skilled Worker
68363,University of Exeter,Exeter,Devon,Worker (A (SME+)),Skilled Worker


In [39]:
var = response.to_dict('records')

[{'Organisation Name': 'University of East Anglia',
  'Town/City': 'Norwich',
  'County': 'Norfolk',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'University of East London',
  'Town/City': 'London',
  'County': nan,
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'University of Edinburgh',
  'Town/City': 'Edinburgh',
  'County': nan,
  'Type & Rating': 'Worker (A (SME+))',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'University of Essex',
  'Town/City': 'Colchester',
  'County': 'Essex',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'University of Exeter',
  'Town/City': 'Exeter',
  'County': 'Devon',
  'Type & Rating': 'Worker (A (SME+))',
  'Route': 'Skilled Worker'}]

In [37]:
import json
from pprint import pprint

json.dumps({str(i) : response.to_dict('records')[i] for i in range(len(response))})


'{"0": {"Organisation Name": "University of East Anglia", "Town/City": "Norwich", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "1": {"Organisation Name": "University of East London", "Town/City": "London", "County": NaN, "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "2": {"Organisation Name": "University of Edinburgh", "Town/City": "Edinburgh", "County": NaN, "Type & Rating": "Worker (A (SME+))", "Route": "Skilled Worker"}, "3": {"Organisation Name": "University of Essex", "Town/City": "Colchester", "County": "Essex", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "4": {"Organisation Name": "University of Exeter", "Town/City": "Exeter", "County": "Devon", "Type & Rating": "Worker (A (SME+))", "Route": "Skilled Worker"}}'

In [31]:
response.to_json()

'{"Organisation Name":{"68351":"University of East Anglia","68353":"University of East London","68355":"University of Edinburgh","68359":"University of Essex","68363":"University of Exeter"},"Town\\/City":{"68351":"Norwich","68353":"London","68355":"Edinburgh","68359":"Colchester","68363":"Exeter"},"County":{"68351":"Norfolk","68353":null,"68355":null,"68359":"Essex","68363":"Devon"},"Type & Rating":{"68351":"Worker (A rating)","68353":"Worker (A rating)","68355":"Worker (A (SME+))","68359":"Worker (A rating)","68363":"Worker (A (SME+))"},"Route":{"68351":"Skilled Worker","68353":"Skilled Worker","68355":"Skilled Worker","68359":"Skilled Worker","68363":"Skilled Worker"}}'

In [20]:
word = "Exis"

{str(i) : search(word).to_dict('records')[i] for i in range(len(search(word)))}

{'0': {'Organisation Name': 'Flexistay Limited',
  'Town/City': 'Croydon',
  'County': 'Surrey',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'}}